# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Santiago Gaviria Zapata
#### Segundo Integrante: Jorge Ricardo Hiler

In [3]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1: Completar código con la librería

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [5]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [77]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

def train(c, kernel, gamma):
    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if(gamma == 0):
            svc_classifier = SVC(C=c,kernel = kernel) 
        else:
            svc_classifier = SVC(C=c,kernel = kernel, gamma=gamma) 
        modelo = svc_classifier.fit(Xtrain, Ytrain)


        #Validación
        Ytrain_pred = svc_classifier.predict(Xtrain)
        Yest = svc_classifier.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1
    return np.mean(EficienciaVal), np.std(EficienciaVal), len(svc_classifier.support_vectors_)/len(Xtrain)
#print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
#print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

## Ejercicio 2: Experimentos

Realice los experimientos necesarios para llenar la siguiente tabla:

In [78]:
import pandas as pd
import qgrid

def fillTable(kernel, C, gamma):
    i = 0
    eficiency = np.zeros(24)
    confidence = np.zeros(24)
    vectors = np.zeros(24)
    for k,c,g in zip(kernel, C, gamma):
        eficiency[i], confidence[i], vectors[i] = train(c,k,g)
        i += 1
    return eficiency, confidence, vectors
        
        
kernel = ['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
C = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gamma = [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]
df_types = pd.DataFrame({
    'Kernel' : pd.Series(kernel),
    'C' : pd.Series(C),
    'gamma' : pd.Series(gamma)})
eficiency, confidence, vectors = fillTable(kernel, C, gamma)
df_types["Eficiencia en validacion"] = eficiency
df_types["Intervalo de confianza"] = confidence
df_types["% de Vectores de Soporte"] = vectors
#df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [79]:
qgrid_widget.get_changed_df()

,Kernel,C,gamma,Eficiencia en validacion,Intervalo de confianza,% de Vectores de Soporte
0,linear,0.001,0.00,0.796001,0.053296,1.000000
1,linear,0.010,0.00,0.970699,0.029265,0.603321
2,linear,0.100,0.00,0.980446,0.016611,0.263838
3,linear,1.000,0.00,0.970777,0.015486,0.254613
4,linear,10.000,0.00,0.970777,0.015486,0.254613
5,linear,100.000,0.00,0.970777,0.015486,0.254613
6,rbf,0.001,0.01,0.315623,0.108064,1.000000
7,rbf,0.001,0.10,0.277914,0.042752,1.000000
8,rbf,0.001,1.00,0.254171,0.001717,0.998155
9,rbf,0.010,0.01,0.315623,0.108064,1.000000


## Ejercicio 3: Completar código sin el uso de la librería

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

#### Nota: No se permite el uso de librerías para resolver este ejercicio

In [71]:
def TrainSVM_OnevsRest(Xtrain, Ytrain, kernel, C, gamma):
    models = []
    support_vectors = []
    for i in range(4):
        YClass = np.where(Ytrain == i, 1, 0)
        if(kernel == 'linear'):
            svc_classifier = SVC(kernel = kernel, C = C, probability = True)   
        else:
            svc_classifier = SVC(kernel = kernel, C = C, gamma = gamma, probability = True)   
        svc_classifier.fit(Xtrain, YClass)
        support_vectors.append(svc_classifier.support_vectors_)
        models.append(svc_classifier)
    return models, support_vectors
    
def ValidaSVM_OnevsRest(X, models):
    n_samples = np.size(X,0)
    n_features = np.size(X,1)
    Yest = np.zeros((n_samples,1))
    n = len(models)
    j = 0
    for xi in X:
        Ypredict = np.zeros(n)
        for i in range(n):
            xi = xi.reshape(1, n_features)
            Yi = models[i].predict_proba(xi)
            Ypredict[i] = Yi[0][1]
        yi_choosen = np.argmax(Ypredict)  
        Yest[j][0] = yi_choosen
        j=j+1
    return Yest

## Ejercicio 4: Entrenamiento

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [74]:
def trainOnevsAll(Kernel, C, gamma):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento

        models, support_vectors = TrainSVM_OnevsRest(Xtrain, Ytrain, Kernel, C , gamma)
        size_support_vectors = []
        for i in range(len(models)):
            size_support_vectors.append(len(support_vectors[i]))

        #Validación
        Ytrain_pred = ValidaSVM_OnevsRest(Xtrain, models)
        Yest = ValidaSVM_OnevsRest(Xtest, models)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1
    return np.mean(EficienciaVal), np.std(EficienciaVal), np.mean(size_support_vectors)/len(Xtrain)


## Ejercicio 5: Experimentos

Realice los experimientos necesarios para llenar la siguiente tabla:

In [75]:
import pandas as pd
import qgrid

def fillTable2(kernel, C, gamma):
    i = 0
    eficiency = np.zeros(24)
    confidence = np.zeros(24)
    vectors = np.zeros(24)
    for k,c,g in zip(kernel, C, gamma):
        eficiency[i], confidence[i], vectors[i] = trainOnevsAll(k, c,g)
        i = i + 1
    return eficiency, confidence, vectors

Kernel = ['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
C = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gamma = [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]
df_types = pd.DataFrame({
    'Kernel' : pd.Series(Kernel),
    'C' : pd.Series(C),
    'gamma' : pd.Series(gamma)})

eficiency, confidence, vectors = fillTable2(Kernel, C, gamma)
df_types["Eficiencia en validacion"] = eficiency
df_types["Intervalo de confianza"] = confidence
df_types["% de Vectores de Soporte"] = vectors
df_types.set_index(['Kernel','C','gamma'], inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [76]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  Intervalo de confianza  \
Kernel C       gamma                                                     
linear 0.001   0.00                   0.953915                0.038728   
       0.010   0.00                   0.962296                0.034566   
       0.100   0.00                   0.974928                0.020473   
       1.000   0.00                   0.977737                0.016372   
       10.000  0.00                   0.977737                0.016372   
       100.000 0.00                   0.977737                0.016372   
rbf    0.001   0.01                   0.963731                0.031997   
               0.10                   0.972259                0.006606   
               1.00                   0.282074                0.015609   
       0.010   0.01                   0.966501                0.029560   
               0.10                   0.919700                0.034911   
               1.00                   0.245767                0.007841   
       0.100   0.01                   0.965097                0.031974   
               0.10                   0.916899                0.032790   
               1.00                   0.245767                0.007841   
       1.000   0.01                   0.981827                0.018834   
               0.10                   0.930804                0.036657   
               1.00                   0.245767                0.007841   
       10.000  0.01                   0.988841                0.010527   
               0.10                   0.941839                0.030498   
               1.00                   0.248568                0.005126   
       100.000 0.01                   0.988841                0.010527   
               0.10                   0.941839                0.030498   
               1.00                   0.248568                0.005126   

                      % de Vectores de Soporte  
Kernel C       gamma                            
linear 0.001   0.00                   0.523524  
       0.010   0.00                   0.287362  
       0.100   0.00                   0.098247  
       1.000   0.00                   0.069188  
       10.000  0.00                   0.069188  
       100.000 0.00                   0.069188  
rbf    0.001   0.01                   0.500923  
               0.10                   0.507841  
               1.00                   0.654059  
       0.010   0.01                   0.544280  
               0.10                   0.976476  
               1.00                   1.000000  
       0.100   0.01                   0.556273  
               0.10                   0.982472  
               1.00                   1.000000  
       1.000   0.01                   0.333026  
               0.10                   0.974170  
               1.00                   1.000000  
       10.000  0.01                   0.279520  
               0.10                   0.970018  
               1.00                   1.000000  
       100.000 0.01                   0.279520  
               0.10                   0.970018  
               1.00                   1.000000

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.

## Ejercicio 6: Comparación

In [86]:
eficiency, confidence, vectors = train(1,'linear',0)
print('Eficiencia durante el validación = ' + str(eficiency) + '\nIntervalo de confiaza = ' + str(confidence) + '\nPorcentaje de vectores de soporte = ' + str(vectors))
eficiency, confidence, vectors = trainOnevsAll('linear',1,0)
print('Eficiencia durante el validación = ' + str(eficiency) + '\nIntervalo de confiaza = ' + str(confidence) + '\nPorcentaje de vectores de soporte = ' + str(vectors))



Eficiencia durante el validación = 0.9707771243055832
Intervalo de confiaza = 0.015485734683194906
Porcentaje de vectores de soporte = 0.25461254612546125
Eficiencia durante el validación = 0.9777370884748747
Intervalo de confiaza = 0.01637228483712346
Porcentaje de vectores de soporte = 0.06918819188191883


### Como podemos observar se obtienen mejores resultados usando la técnica de uno vs el resto, pese a que el porcentaje de vectores de soporte sea menor 